In [ ]:
# -- Install and setup Oracle Instant Client --
! wget -O oracleinstantclient.zip https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
! unzip -oq oracleinstantclient.zip
! mkdir -p /opt/oracle
! mv instantclient_19_6 /opt/oracle/
! sh -c "echo /opt/oracle/instantclient_19_6 > /etc/ld.so.conf.d/oracle-instantclient.conf"
! ldconfig

# -- Include cx_Oracle package --
! pip install cx_Oracle

--2025-04-29 14:37:58--  https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
Resolving download.oracle.com (download.oracle.com)... 23.192.208.88
Connecting to download.oracle.com (download.oracle.com)|23.192.208.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75462547 (72M) [application/zip]
Saving to: ‘oracleinstantclient.zip’

oracleinstantclient 100%[===================>]  71.97M  91.8MB/s    in 0.8s    

2025-04-29 14:37:59 (91.8 MB/s) - ‘oracleinstantclient.zip’ saved [75462547/75462547]

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_level_zero.so.0 is not a symbolic link

/sbin/ldco

In [ ]:
from google.colab.widgets import TabBar
import ipywidgets as widgets
from ipywidgets import Label, Text, Dropdown, Button, VBox, HBox, Textarea, IntText, FloatText, DatePicker, HTML, Output
import cx_Oracle
from datetime import date, datetime
import re
import pandas as pd

# Setup database connection configuration
HOST_NAME = "imz409.ust.hk"
PORT_NUMBER = "1521"
SERVICE_NAME = "imz409"
USERNAME = "LA302"
PASSWORD = "CJ995"

# Establish connection to the Oracle database
try:
    dsn_tns = cx_Oracle.makedsn(HOST_NAME, PORT_NUMBER, service_name=SERVICE_NAME)
    conn = cx_Oracle.connect(user=USERNAME, password=PASSWORD, dsn=dsn_tns)
    c = conn.cursor()
    print("Database connection established successfully.")
except cx_Oracle.DatabaseError as e:
    error_obj, = e.args
    print(f"Database connection failed: {error_obj.message}")

# Create the TabBar with specified tabs
tb = TabBar([
    'Welcome',
    'Admin Login',
    'Admin Menu',
    'Adm: Add Officer',
    'Adm: View Officers',
    'Adm: Update Officer',
    'Adm: Add Product',
    'Adm: View Product',
    'Adm: Edit Product',
    'Adm: Proceed Order',
    'Adm: Generate Report',
    'Adm: Orders Today',
    'Adm: Manager Click Report',
    'Officer Login',
    'Officer: View and Process TradeIns'
])


########################################
############ Welcome Tab ##############
########################################
with tb.output_to('Welcome'):
    # Welcome message
    welcome_label = Label("Welcome to the Staff Management System", style={'font_weight': 'bold', 'font_size': '20px'})
    # Button to navigate to Admin Login
    go_to_login_button = Button(description="Admin Login", button_style='info', layout={'width': '150px', 'margin': '10px'})
    btnOfficer = Button(description="Officer Login",button_style='info', layout={'width': '150px', 'margin': '10px'})

    # Function to switch to Admin Login tab
    def on_go_to_login_click(b):
        showPage('Admin Login')

    go_to_login_button.on_click(on_go_to_login_click)

    def on_btnOfficer_clicked(b):
      showPage("Officer Login")
    btnOfficer.on_click(on_btnOfficer_clicked)


    # Display the Welcome tab content
    display(VBox([
        welcome_label,
        go_to_login_button,
        btnOfficer
    ], layout={'align_items': 'center', 'justify_content': 'center', 'width': '100%'}))

########################################
############ Admin Login ###############
########################################
with tb.output_to('Admin Login'):
    login_label = Label("Admin Login", style={'font_weight': 'bold', 'font_size': '20px'})
    username_input = Text(description="Username:", placeholder="Enter your username", layout={'width': '350px', 'margin': '5px'})
    password_input = Text(description="Password:", placeholder="Enter your password", layout={'width': '350px', 'margin': '5px'}, style={'font_weight': 'bold'}, type='password')
    login_status_label = Label(layout={'width': '350px', 'margin': '5px'})
    login_button = Button(description="Login", button_style='info', layout={'width': '150px', 'margin': '5px'})
    forget_password_button = Button(description="Forget Password", button_style='danger', layout={'width': '200px', 'margin': '5px'})

    # Global variable to store logged-in admin information
    global logged_in_admin
    logged_in_admin = None

    # Functionality for Admin Login
    def login(b):
        try:
            if not username_input.value or not password_input.value:
                login_status_label.value = "❗ Both username and password are required."
                return

            # Validate credentials using the STAFF table (Admin type accounts)
            c.execute("""
                SELECT S_ID, S_FIRST_NAME, S_LAST_NAME, S_USERNAME, S_EMAIL, S_TYPE
                FROM STAFF
                WHERE S_USERNAME = :1 AND S_PW = :2 AND S_TYPE = 'Admin'
            """, [username_input.value, password_input.value])
            admin = c.fetchone()

            if admin:
                # Populate the logged_in_admin dictionary
                global logged_in_admin
                logged_in_admin = {
                    'id': admin[0],
                    'first_name': admin[1],
                    'last_name': admin[2],
                    'username': admin[3],
                    'email': admin[4],
                    'role': admin[5]
                }

                login_status_label.value = f"✅ Welcome, {admin[1]} {admin[2]} ({admin[5]}). Redirecting to Admin Menu..."
                # Redirect to Admin Menu
                showPage('Admin Menu')
            else:
                login_status_label.value = "❌ Invalid username or password. Please try again."

        except cx_Oracle.DatabaseError as e:
            error_obj, = e.args
            login_status_label.value = f"❌ Database Error: {error_obj.message}"

    # Functionality for Forget Password
    def forget_password(b):
        login_status_label.value = "🔒 Please go to the IT officer with your staff card to request a password reset."

    # Bind the functions to the buttons
    login_button.on_click(login)
    forget_password_button.on_click(forget_password)

    # Display the Admin Login interface
    display(VBox([
        login_label,
        username_input,
        password_input,
        HBox([login_button, forget_password_button]),  # Buttons displayed side by side
        login_status_label
    ]))

########################################
########### Admin Menu Tab ############
########################################
def showPage(tabName):
    with tb.output_to(tabName):
        display()

with tb.output_to('Adm: Orders Today'):
    orders_today_label = Label("Orders Received Today", style={'font_weight': 'bold', 'font_size': '20px'})
    orders_table_output = HTML(value="<p>Loading orders...</p>")
    orders_back_button = Button(description="Back to Admin Menu", layout={'width': '250px', 'margin': '10px'}, button_style="primary")

    # Function to fetch and display today's orders
    def load_orders_today():
        try:
            # Fetch today's orders
            c.execute("""
                SELECT OR_ID, OR_STATUS, OR_DATE, OR_SHIPDATE, OR_ESTIMATED_DAY, OR_REMARK
                FROM ORDERS
                WHERE TRUNC(OR_DATE) = TRUNC(SYSDATE)
                ORDER BY OR_DATE
            """)
            rows = c.fetchall()
            if not rows:
                orders_table_output.value = "<p>No orders received today.</p>"
                return

            # Create an HTML table for orders
            html = "<style>table {border-collapse: collapse; width: 100%;} td, th {border: 1px solid #ddd; padding: 8px;}</style>"
            html += "<table><tr><th>Order ID</th><th>Status</th><th>Order Date</th><th>Ship Date</th><th>Estimated Delivery</th><th>Remarks</th></tr>"
            for row in rows:
                html += f"<tr><td>{row[0]}</td><td>{row[1]}</td><td>{row[2]}</td><td>{row[3] if row[3] else 'N/A'}</td><td>{row[4] if row[4] else 'N/A'}</td><td>{row[5]}</td></tr>"
            html += "</table>"
            orders_table_output.value = html
        except cx_Oracle.DatabaseError as e:
            error_obj, = e.args
            orders_table_output.value = f"<p>❌ Database Error: {error_obj.message}</p>"

    # Load orders when the tab is rendered
    load_orders_today()

    # Back to Admin Menu button
    def on_back_to_admin_menu_click(b):
        showPage('Admin Menu')

    orders_back_button.on_click(on_back_to_admin_menu_click)

    # Display orders page
    display(VBox([orders_today_label, orders_table_output, orders_back_button],
                 layout={'align_items': 'center', 'justify_content': 'center', 'width': '100%'}))



with tb.output_to('Admin Menu'):
    # Admin Menu Header
    admin_label = Label("Admin Menu", style={'font_weight': 'bold', 'font_size': '20px'})


    # Dashboard Items
    orders_today_label = Label("Orders Received Today:", style={'font_weight': 'bold', 'font_size': '16px'})
    orders_today_value = Label("Loading...", style={'font_size': '16px'})
    view_orders_today_button = Button(description="View All Orders Today", layout={'width': '250px', 'margin': '10px'}, button_style="primary")

    revenue_month_label = Label("Revenue This Month (Current Calendar):", style={'font_weight': 'bold', 'font_size': '16px'})
    revenue_month_value = Label("Loading...", style={'font_size': '16px'})

    top_items_label = Label("Top 5 Popular Items (by Units Sold):", style={'font_weight': 'bold', 'font_size': '16px'})
    top_items_value = HTML("<p>Loading...</p>", style={'font_size': '16px'})

    revenue_today_label = Label("Sales Revenue for Today:", style={'font_weight': 'bold', 'font_size': '16px'})
    revenue_today_value = Label("Loading...", style={'font_size': '16px'})

        # Fetch data for the dashboard
    def load_dashboard_data():
        try:
            # Fetch number of orders received today
            c.execute("""
                SELECT COUNT(*)
                FROM ORDERS
                WHERE TRUNC(OR_DATE) = TRUNC(SYSDATE)
            """)
            orders_today = c.fetchone()[0]
            orders_today_value.value = f"{orders_today} orders"

            # Fetch revenue for the current calendar month
            c.execute("""
                SELECT SUM(OL_SUBTOTAL)
                FROM ORDERS
                JOIN ORDERLINE ON ORDERS.OR_ID = ORDERLINE.ORDERS_OR_ID
                WHERE TO_CHAR(OR_DATE, 'YYYY-MM') = TO_CHAR(SYSDATE, 'YYYY-MM')
            """)
            revenue_month = c.fetchone()[0]
            revenue_month_value.value = f"${revenue_month:.2f}" if revenue_month else "$0.00"

            # Fetch top 5 popular items by units sold
            c.execute("""
                SELECT P_NAME, SUM(OL_P_QTY) AS TOTAL_UNITS
                FROM ORDERLINE
                JOIN PRODUCT ON ORDERLINE.PRODUCT_P_ID = PRODUCT.P_ID
                GROUP BY P_NAME
                ORDER BY TOTAL_UNITS DESC FETCH FIRST 5 ROWS ONLY
            """)
            top_items = c.fetchall()
            if top_items:
                top_items_html = "<ul>"
                for item in top_items:
                    top_items_html += f"<li>{item[0]}: {item[1]} units</li>"
                top_items_html += "</ul>"
                top_items_value.value = top_items_html
            else:
                top_items_value.value = "<p>No sales data available.</p>"

            # Fetch sales revenue for today
            c.execute("""
                SELECT SUM(OL_SUBTOTAL)
                FROM ORDERS
                JOIN ORDERLINE ON ORDERS.OR_ID = ORDERLINE.ORDERS_OR_ID
                WHERE TRUNC(OR_DATE) = TRUNC(SYSDATE)
            """)
            revenue_today = c.fetchone()[0]
            revenue_today_value.value = f"${revenue_today:.2f}" if revenue_today else "$0.00"

        except cx_Oracle.DatabaseError as e:
            error_obj, = e.args
            orders_today_value.value = f"❌ Error: {error_obj.message}"
            revenue_month_value.value = f"❌ Error: {error_obj.message}"
            top_items_value.value = f"<p>❌ Error: {error_obj.message}</p>"
            revenue_today_value.value = f"❌ Error: {error_obj.message}"

    # Original Buttons
    add_officer_btn = Button(description="Add Officer", layout={'width': '250px', 'margin': '10px'}, button_style="primary")
    view_officers_btn = Button(description="View Officers", layout={'width': '250px', 'margin': '10px'}, button_style="info")
    update_officer_btn = Button(description="Update Officer", layout={'width': '250px', 'margin': '10px'}, button_style="warning")
    add_product_btn = Button(description="Add Product", layout={'width': '250px', 'margin': '10px'}, button_style="primary")
    view_product_btn = Button(description="View Product", layout={'width': '250px', 'margin': '10px'}, button_style="info")
    edit_product_btn = Button(description="Edit Product", layout={'width': '250px', 'margin': '10px'}, button_style="warning")
    proceed_order_btn = Button(description="Proceed Order", layout={'width': '250px', 'margin': '10px'}, button_style="primary")
    gen_report_btn = Button(description="Generate Report", layout={'width': '250px', 'margin': '10px'}, button_style="info")
    view_top_clck_btn= Button(description="Manager Click Report", layout={'width': '250px', 'margin': '10px'}, button_style="primary")
    logout_button = Button(description="Logout", button_style='danger', layout={'width': '200px', 'margin': '5px'})

    # Dashboard Items
    orders_today_label = Label("Orders Received Today:", style={'font_weight': 'bold', 'font_size': '16px'})
    orders_today_value = Label("Loading...", style={'font_size': '16px'})
    view_orders_today_button = Button(description="View All Orders Today", layout={'width': '250px', 'margin': '10px'}, button_style="primary")

    revenue_month_label = Label("Revenue This Month (Current Calendar):", style={'font_weight': 'bold', 'font_size': '16px'})
    revenue_month_value = Label("Loading...", style={'font_size': '16px'})

    top_items_label = Label("Top 5 Popular Items (by Units Sold):", style={'font_weight': 'bold', 'font_size': '16px'})
    top_items_value = HTML("<p>Loading...</p>", style={'font_size': '16px'})

    revenue_today_label = Label("Sales Revenue for Today:", style={'font_weight': 'bold', 'font_size': '16px'})
    revenue_today_value = Label("Loading...", style={'font_size': '16px'})

    # Fetch data for the dashboard
    def load_dashboard_data():
        try:
            # Fetch number of orders received today
            c.execute("""
                SELECT COUNT(*)
                FROM ORDERS
                WHERE TRUNC(OR_DATE) = TRUNC(SYSDATE)
            """)
            orders_today = c.fetchone()[0]
            orders_today_value.value = f"{orders_today} orders"

            # Fetch revenue for the current calendar month
            c.execute("""
                SELECT SUM(OL_SUBTOTAL)
                FROM ORDERS
                JOIN ORDERLINE ON ORDERS.OR_ID = ORDERLINE.ORDERS_OR_ID
                WHERE TO_CHAR(OR_DATE, 'YYYY-MM') = TO_CHAR(SYSDATE, 'YYYY-MM')
            """)
            revenue_month = c.fetchone()[0]
            revenue_month_value.value = f"${revenue_month:.2f}" if revenue_month else "$0.00"

            # Fetch top 5 popular items by units sold
            c.execute("""
                SELECT P_NAME, SUM(OL_P_QTY) AS TOTAL_UNITS
                FROM ORDERLINE
                JOIN PRODUCT ON ORDERLINE.PRODUCT_P_ID = PRODUCT.P_ID
                GROUP BY P_NAME
                ORDER BY TOTAL_UNITS DESC FETCH FIRST 5 ROWS ONLY
            """)
            top_items = c.fetchall()
            if top_items:
                top_items_html = "<ul>"
                for item in top_items:
                    top_items_html += f"<li>{item[0]}: {item[1]} units</li>"
                top_items_html += "</ul>"
                top_items_value.value = top_items_html
            else:
                top_items_value.value = "<p>No sales data available.</p>"

            # Fetch sales revenue for today
            c.execute("""
                SELECT SUM(OL_SUBTOTAL)
                FROM ORDERS
                JOIN ORDERLINE ON ORDERS.OR_ID = ORDERLINE.ORDERS_OR_ID
                WHERE TRUNC(OR_DATE) = TRUNC(SYSDATE)
            """)
            revenue_today = c.fetchone()[0]
            revenue_today_value.value = f"${revenue_today:.2f}" if revenue_today else "$0.00"

        except cx_Oracle.DatabaseError as e:
            error_obj, = e.args
            orders_today_value.value = f"❌ Error: {error_obj.message}"
            revenue_month_value.value = f"❌ Error: {error_obj.message}"
            top_items_value.value = f"<p>❌ Error: {error_obj.message}</p>"
            revenue_today_value.value = f"❌ Error: {error_obj.message}"

    # Button click handlers to switch tabs
    def on_add_officer_click(b):
        showPage('Adm: Add Officer')

    def on_view_officers_click(b):
        showPage('Adm: View Officers')

    def on_update_officer_click(b):
        showPage('Adm: Update Officer')

    def on_add_product_click(b):
        showPage('Adm: Add Product')

    def on_view_product_click(b):
        showPage('Adm: View Product')

    def on_edit_product_click(b):
        showPage('Adm: Edit Product')

    def on_proceed_order_click(b):
        showPage('Adm: Proceed Order')

    def on_gen_report_click(b):
        showPage('Adm: Generate Report')

    def on_view_orders_today_click(b):
        showPage('Adm: Orders Today')  # Separate page for today's orders

    def on_view_top_clck_click(b):
        showPage('Adm: Manager Click Report')

    def on_logout_clicked(b):
        # Clear the login fields
        username_input.value = ""
        password_input.value = ""
        login_status_label.value = ""  # Clear any previous login status messages

        # Redirect to the Admin Login page
        showPage('Admin Login')

    # Attach button handlers
    add_officer_btn.on_click(on_add_officer_click)
    view_officers_btn.on_click(on_view_officers_click)
    update_officer_btn.on_click(on_update_officer_click)
    add_product_btn.on_click(on_add_product_click)
    view_product_btn.on_click(on_view_product_click)
    edit_product_btn.on_click(on_edit_product_click)
    proceed_order_btn.on_click(on_proceed_order_click)
    gen_report_btn.on_click(on_gen_report_click)
    view_orders_today_button.on_click(on_view_orders_today_click)
    view_top_clck_btn.on_click(on_view_top_clck_click)
    logout_button.on_click(on_logout_clicked)

    # Load dashboard data
    load_dashboard_data()

    # Display Admin Menu with Dashboard
    display(VBox([
        admin_label,
        orders_today_label, orders_today_value, view_orders_today_button,
        revenue_today_label, revenue_today_value,
        revenue_month_label, revenue_month_value,
        top_items_label, top_items_value,
        add_officer_btn, view_officers_btn, update_officer_btn,
        add_product_btn, view_product_btn, edit_product_btn,
        proceed_order_btn, gen_report_btn,view_top_clck_btn,
        logout_button
    ], layout={'align_items': 'center', 'justify_content': 'center', 'width': '100%'}))



########################################
########## Adm: Add Officer ###########
########################################
with tb.output_to('Adm: Add Officer'):
    add_label = Label("Add New Officer", style={'font_weight': 'bold', 'font_size': '18px'})
    add_username = Text(description="Username:", placeholder="3-30 alphanumeric characters", layout={'width': '350px', 'margin': '5px'})
    add_firstname = Text(description="First Name:", placeholder="2-50 letters only", layout={'width': '350px', 'margin': '5px'})
    add_lastname = Text(description="Last Name:", placeholder="2-50 letters only", layout={'width': '350px', 'margin': '5px'})
    add_password = Text(description="Password:", placeholder="6-30 characters", layout={'width': '350px', 'margin': '5px'})
    add_email = Text(description="Email:", placeholder="Valid email format", layout={'width': '350px', 'margin': '5px'})
    add_status_label = Label(layout={'width': '350px', 'margin': '5px'})
    add_button = Button(description="Add Officer", button_style='success', layout={'width': '150px', 'margin': '5px'})
    clear_button = Button(description="Clear Form", layout={'width': '150px', 'margin': '5px'})
    add_back_button = widgets.Button(description="Back",layout={'width': '150px', 'margin': '5px'})

    # Helper functions for validation
    def is_valid_username(username):
        return re.match(r"^[a-zA-Z0-9]{3,30}$", username)

    def is_valid_email(email):
        return re.match(r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$", email)

    def is_valid_name(name):
        return re.match(r"^[a-zA-Z]{2,50}$", name)

    def is_valid_password(password):
        return 6 <= len(password) <= 30

    # Add Officer logic
    def on_add_button_clicked(b):
        try:
            if not all([add_username.value, add_firstname.value, add_lastname.value, add_password.value, add_email.value]):
                add_status_label.value = "❗ All fields are required."
                return

            if not is_valid_username(add_username.value):
                add_status_label.value = "❗ Invalid username. Use 3-30 alphanumeric characters."
                return

            if not is_valid_name(add_firstname.value):
                add_status_label.value = "❗ Invalid first name. Use 2-50 letters only."
                return

            if not is_valid_name(add_lastname.value):
                add_status_label.value = "❗ Invalid last name. Use 2-50 letters only."
                return

            if not is_valid_password(add_password.value):
                add_status_label.value = "❗ Invalid password. Use 6-30 characters."
                return

            if not is_valid_email(add_email.value):
                add_status_label.value = "❗ Invalid email format."
                return

            # Check if username already exists
            c.execute("SELECT COUNT(*) FROM STAFF WHERE S_USERNAME = :1", [add_username.value])
            if c.fetchone()[0] > 0:
                add_status_label.value = "❗ Username already exists."
                return

            # Insert officer into database
            c.execute("""
                INSERT INTO STAFF (S_ID, S_USERNAME, S_FIRST_NAME, S_LAST_NAME, S_PW, S_TYPE, S_STATUS, S_EMAIL)
                VALUES (STAFF_SEQ.NEXTVAL, :1, :2, :3, :4, 'Officer', 'Active', :5)
            """, [add_username.value, add_firstname.value, add_lastname.value, add_password.value, add_email.value])
            conn.commit()
            add_status_label.value = "✅ Officer added successfully."
        except cx_Oracle.DatabaseError as e:
            error_obj, = e.args
            add_status_label.value = f"❌ Database Error: {error_obj.message}"

    add_button.on_click(on_add_button_clicked)

    def on_clear_button_clicked(b):
        add_username.value = ""
        add_firstname.value = ""
        add_lastname.value = ""
        add_password.value = ""
        add_email.value = ""
        add_status_label.value = ""

    clear_button.on_click(on_clear_button_clicked)

    def on_add_back_button_clicked(b):
      showPage('Admin Menu')
    add_back_button.on_click(on_add_back_button_clicked)

    display(VBox([add_label, add_username, add_firstname, add_lastname, add_password, add_email,
                  HBox([add_button, clear_button, add_back_button]), add_status_label]))

########################################
########## Adm: View Officers #########
########################################
with tb.output_to('Adm: View Officers'):
    view_label = Label("View Officers", style={'font_weight': 'bold', 'font_size': '18px'})
    refresh_button = Button(description="Refresh List", button_style='info', layout={'width': '150px', 'margin': '5px'})
    search_username = Text(description="Search Username:", placeholder="Enter username to search", layout={'width': '350px', 'margin': '5px'})
    search_button = Button(description="Search", button_style='info', layout={'width': '100px', 'margin': '5px'})
    officers_table = HTML(value="<p>Click 'Refresh List' to load officers</p>")
    total_officers_label = Label(value="", layout={'width': '350px', 'margin': '5px'})  # Label to show total officers count
    view_back_button = widgets.Button(description="Back", layout={'width': '150px', 'margin': '5px'})

    def load_officers(search_term=None):
        try:
            if search_term:
                # Query to search officers by username
                c.execute("""
                    SELECT S_ID, S_USERNAME, S_FIRST_NAME, S_LAST_NAME, S_STATUS, S_EMAIL
                    FROM STAFF
                    WHERE S_TYPE = 'Officer' AND S_USERNAME LIKE :1
                    ORDER BY S_USERNAME
                """, [f"%{search_term}%"])
            else:
                # Query to fetch all officers ordered by username
                c.execute("""
                    SELECT S_ID, S_USERNAME, S_FIRST_NAME, S_LAST_NAME, S_STATUS, S_EMAIL
                    FROM STAFF
                    WHERE S_TYPE = 'Officer'
                    ORDER BY S_USERNAME
                """)

            rows = c.fetchall()
            if not rows:
                officers_table.value = "<p>No officers found in the system.</p>"
                total_officers_label.value = "Total Officers: 0"

            # Generate the HTML table
            html = "<style>table {border-collapse: collapse; width: 100%;} td, th {border: 1px solid #ddd; padding: 8px;}</style>"
            html += "<table><tr><th>#</th><th>ID</th><th>Username</th><th>First Name</th><th>Last Name</th><th>Email</th><th>Status</th></tr>"
            for idx, row in enumerate(rows, start=1):  # Add index column
                html += f"""
                <tr>
                    <td>{idx}</td>  <!-- Index -->
                    <td>{row[0]}</td>
                    <td>{row[1]}</td>
                    <td>{row[2]}</td>
                    <td>{row[3]}</td>
                    <td>{row[5]}</td>
                    <td>{row[4]}</td>
                </tr>
                """
            html += "</table>"

            # Set the table and total officers count
            officers_table.value = html
            total_officers_label.value = f"Total Officers: {len(rows)}"

            return
        except cx_Oracle.DatabaseError as e:
            error_obj, = e.args
            officers_table.value = f"<p>❌ Database Error: {error_obj.message}</p>"
            total_officers_label.value = ""

    def on_refresh_button_clicked(b):
        search_username.value = ""
        load_officers()

    def on_search_button_clicked(b):
        load_officers(search_username.value)

    def on_view_back_button_clicked(b):
        showPage('Admin Menu')

    view_back_button.on_click(on_view_back_button_clicked)
    refresh_button.on_click(on_refresh_button_clicked)
    search_button.on_click(on_search_button_clicked)

    display(VBox([
        view_label,
        HBox([search_username, search_button]),
        refresh_button,
        total_officers_label,  # Show total officers count
        officers_table,        # Table showing the officers
        HBox([view_back_button])
    ]))

########################################
######### Adm: Update Officer #########
########################################
with tb.output_to('Adm: Update Officer'):
    update_label = Label("Update Officer Information", style={'font_weight': 'bold', 'font_size': '18px'})
    officer_id_input = IntText(description="Officer ID:", layout={'width': '350px', 'margin': '5px'})
    fetch_button = Button(description="Fetch Officer", button_style='info', layout={'width': '150px', 'margin': '5px'})

    update_username = Text(description="Username:", disabled=True, layout={'width': '350px', 'margin': '5px'})
    update_firstname = Text(description="First Name:", layout={'width': '350px', 'margin': '5px'})
    update_lastname = Text(description="Last Name:", layout={'width': '350px', 'margin': '5px'})
    update_email = Text(description="Email:", layout={'width': '350px', 'margin': '5px'})
    update_status = Dropdown(description="Status:", options=['Active', 'Inactive'], layout={'width': '350px', 'margin': '5px'})

    update_button = Button(description="Update Officer", button_style='success', disabled=True, layout={'width': '150px', 'margin': '5px'})
    clear_update_button = Button(description="Clear Form", layout={'width': '150px', 'margin': '5px'})
    update_status_label = Label(layout={'width': '350px', 'margin': '5px'})
    update_back_button = widgets.Button(description="Back",layout={'width': '150px', 'margin': '5px'})

    def on_fetch_button_clicked(b):
        try:
            officer_id = officer_id_input.value
            if not officer_id:
                update_status_label.value = "❗ Please enter an Officer ID."
                return

            c.execute("""
                SELECT S_USERNAME, S_FIRST_NAME, S_LAST_NAME, S_EMAIL, S_STATUS
                FROM STAFF WHERE S_ID = :1 AND S_TYPE = 'Officer'
            """, [officer_id])

            officer_data = c.fetchone()
            if not officer_data:
                update_status_label.value = "❗ No officer found with this ID."
                update_button.disabled = True
                return

            update_username.value = officer_data[0]
            update_firstname.value = officer_data[1]
            update_lastname.value = officer_data[2]
            update_email.value = officer_data[3]
            update_status.value = officer_data[4]
            update_button.disabled = False
            update_status_label.value = "✅ Officer data loaded. You can now update."

        except cx_Oracle.DatabaseError as e:
            error_obj, = e.args
            update_status_label.value = f"❌ Database Error: {error_obj.message}"

    def on_update_button_clicked(b):
        try:
            if not all([update_firstname.value, update_lastname.value, update_email.value]):
                update_status_label.value = "❗ All fields are required."
                return

            if not is_valid_name(update_firstname.value):
                update_status_label.value = "❗ Invalid first name. Use 2-50 letters only."
                return

            if not is_valid_name(update_lastname.value):
                update_status_label.value = "❗ Invalid last name. Use 2-50 letters only."
                return

            if not is_valid_email(update_email.value):
                update_status_label.value = "❗ Invalid email format."
                return

            c.execute("""
                UPDATE STAFF
                SET S_FIRST_NAME = :1, S_LAST_NAME = :2, S_EMAIL = :3, S_STATUS = :4
                WHERE S_ID = :5
            """, [update_firstname.value, update_lastname.value, update_email.value,
                 update_status.value, officer_id_input.value])
            conn.commit()
            update_status_label.value = "✅ Officer information updated successfully."

        except cx_Oracle.DatabaseError as e:
            error_obj, = e.args
            update_status_label.value = f"❌ Database Error: {error_obj.message}"

    def on_clear_update_button_clicked(b):
        officer_id_input.value = 0
        update_username.value = ""
        update_firstname.value = ""
        update_lastname.value = ""
        update_email.value = ""
        update_status.value = "Active"
        update_button.disabled = True
        update_status_label.value = ""

    def on_update_back_button_clicked(b):
      showPage('Admin Menu')
    update_back_button.on_click(on_update_back_button_clicked)

    fetch_button.on_click(on_fetch_button_clicked)
    update_button.on_click(on_update_button_clicked)
    clear_update_button.on_click(on_clear_update_button_clicked)

    display(VBox([
        update_label,
        officer_id_input,
        fetch_button,
        update_username,
        update_firstname,
        update_lastname,
        update_email,
        update_status,
        HBox([update_button, clear_update_button,update_back_button]),
        update_status_label
    ]))

########################################
########## Adm: Add Product ###########
########################################
with tb.output_to('Adm: Add Product'):
    add_product_label = Label("Add New Product", style={'font_weight': 'bold', 'font_size': '18px'})
    product_name = Text(description="Product Name:", placeholder="Enter product name", layout={'width': '350px', 'margin': '5px'})
    product_price = FloatText(description="Price (HKD):", min=0, layout={'width': '350px', 'margin': '5px'})
    product_description = Textarea(description="Description:", placeholder="Enter product description", layout={'width': '350px', 'margin': '5px'})
    product_color = Text(description="Color:", placeholder="Enter product color", layout={'width': '350px', 'margin': '5px'})
    product_capacity = Text(description="Capacity:", placeholder="Enter product capacity", layout={'width': '350px', 'margin': '5px'})
    product_stock = IntText(description="Stock Quantity:", min=0, layout={'width': '350px', 'margin': '5px'})
    product_sku = Text(description="SKU:", placeholder="Enter product SKU (xxxx-xxxx-xxxx)", layout={'width': '350px', 'margin': '5px'})
    product_status = Dropdown(description="Status:", options=['Available', 'Out of Stock', 'Discontinued'], layout={'width': '350px', 'margin': '5px'})

    # Dropdown for Product Category
    product_category = Dropdown(
        description="Category:",
        options=["Smartphone", "Tablet", "Laptop", "Desktop", "Accessory", "Other"],
        layout={'width': '350px', 'margin': '5px'}
    )

    add_product_button = Button(description="Add Product", button_style='success', layout={'width': '150px', 'margin': '5px'})
    clear_product_button = Button(description="Clear Form", layout={'width': '150px', 'margin': '5px'})
    product_status_label = Label(layout={'width': '1000px', 'margin': '5px'})
    add_product_back_button = widgets.Button(description="Back", layout={'width': '150px', 'margin': '5px'})

    def on_add_product_button_clicked(b):
        try:
            # Check the blank field
            if not all([product_name.value, product_description.value, product_color.value,
                       product_capacity.value, product_sku.value]):
                product_status_label.value = "❗ All fields are required."
                return

            # Check the length of product name
            if len(product_name.value) < 2:
                product_status_label.value = "❗ Product name must be at least 2 characters."
                return

            if len(product_name.value) > 50:
                product_status_label.value = "❗ Product name cannot exceed 50 characters."
                return

            # Check the value of product price
            if product_price.value <= 0:
                product_status_label.value = "❗ Price must be greater than 0."
                return

            # Check the length of product description
            if len(product_description.value) < 10:
                product_status_label.value = "❗ Description must be at least 10 characters."
                return

            if len(product_description.value) > 400:
                product_status_label.value = "❗ Description cannot exceed 400 characters."
                return

            # Check the length of product color
            if len(product_color.value) < 2:
                product_status_label.value = "❗ Color must be at least 2 characters."
                return

            if len(product_color.value) > 20:
                product_status_label.value = "❗ Color cannot exceed 20 characters."
                return

            # Check the length of product capacity
            if len(product_capacity.value) < 2:
                product_status_label.value = "❗ Capacity must be at least 2 characters."
                return

            if len(product_capacity.value) > 20:
                product_status_label.value = "❗ Capacity cannot exceed 20 characters."
                return

            # Check the value range of stock
            if product_stock.value < 0:
                product_status_label.value = "❗ Stock quantity cannot be negative."
                return

            # Check whether SKU fulfills the predetermined format
            import re

            def validate_sku(sku):
                return re.match(r"^(?=.{5,50}$)[A-Za-z0-9]+(?:-[A-Za-z0-9]+){2}$", sku)

            if not validate_sku(product_sku.value):
                product_status_label.value = "❗ Invalid SKU."
                return

            # Check if SKU already exists
            c.execute("SELECT COUNT(*) FROM PRODUCT WHERE P_SKU = :1", [product_sku.value])
            if c.fetchone()[0] > 0:
                product_status_label.value = "❗ SKU already exists."
                return

            # Insert product into database
            c.execute("""
                INSERT INTO PRODUCT (
                    P_ID, P_NAME, P_PRICE, P_DESCRIPTION, P_COLOR, P_CAPACITY,
                    P_STOCK_QTY, P_SKU, P_STATUS, P_CATEGORY, P_DATE_ADDED
                ) VALUES (
                    PRODUCT_SEQ.NEXTVAL, :1, :2, :3, :4, :5, :6, :7, :8, :9, SYSDATE
                )
            """, [
                product_name.value, product_price.value, product_description.value,
                product_color.value, product_capacity.value, product_stock.value,
                product_sku.value, product_status.value, product_category.value
            ])
            conn.commit()
            product_status_label.value = "✅ Product added successfully."
        except cx_Oracle.DatabaseError as e:
            error_obj, = e.args
            product_status_label.value = f"❌ Database Error: {error_obj.message}"

    def on_clear_product_button_clicked(b):
        product_name.value = ""
        product_price.value = 0
        product_description.value = ""
        product_color.value = ""
        product_capacity.value = ""
        product_stock.value = 0
        product_sku.value = ""
        product_status.value = "Available"
        product_category.value = "Smartphone"
        product_status_label.value = ""

    def on_add_product_back_button_clicked(b):
        showPage('Admin Menu')

    add_product_back_button.on_click(on_add_product_back_button_clicked)
    add_product_button.on_click(on_add_product_button_clicked)
    clear_product_button.on_click(on_clear_product_button_clicked)

    display(VBox([
        add_product_label,
        product_name,
        product_price,
        product_description,
        product_color,
        product_capacity,
        product_stock,
        product_sku,
        product_status,
        product_category,  # Updated: Dropdown for category
        HBox([add_product_button, clear_product_button, add_product_back_button]),
        product_status_label
    ]))

########################################
########## Adm: View Product ##########
########################################
with tb.output_to('Adm: View Product'):
    view_product_label = Label("View Products", style={'font_weight': 'bold', 'font_size': '18px'})
    product_refresh_button = Button(description="Refresh List", button_style='info', layout={'width': '150px', 'margin': '5px'})
    search_product_id = IntText(description="Search by ID:", layout={'width': '200px', 'margin': '5px'})
    search_product_name = Text(description="Search by Name:", placeholder="Enter product name", layout={'width': '350px', 'margin': '5px'})
    search_product_button = Button(description="Search", button_style='info', layout={'width': '100px', 'margin': '5px'})
    products_table = HTML(value="<p>Click 'Refresh List' to load products</p>")
    product_search_status = Label(layout={'width': '350px', 'margin': '5px'})
    total_products_label = Label(value="", style={'font_weight': 'bold', 'font_size': '16px', 'margin': '5px'})
    view_product_back_button = widgets.Button(description="Back",layout={'width': '150px', 'margin': '5px'})

    def load_products(product_id=None, product_name=None):
        try:
            if product_id:
                # Query to fetch product by ID within allowed categories
                c.execute("""
                    SELECT P_ID, P_NAME, P_PRICE, P_STOCK_QTY, P_SKU, P_STATUS, P_CATEGORY, P_DATE_ADDED
                    FROM PRODUCT
                    WHERE P_ID = :1 AND P_CATEGORY IN ('Smartphone', 'Tablet', 'Laptop', 'Desktop', 'Accessory')
                    ORDER BY P_ID
                """, [product_id])
            elif product_name:
                # Query to fetch products by name within allowed categories
                c.execute("""
                    SELECT P_ID, P_NAME, P_PRICE, P_STOCK_QTY, P_SKU, P_STATUS, P_CATEGORY, P_DATE_ADDED
                    FROM PRODUCT
                    WHERE UPPER(P_NAME) LIKE UPPER(:1) AND P_CATEGORY IN ('Smartphone', 'Tablet', 'Laptop', 'Desktop', 'Accessory')
                    ORDER BY P_ID
                """, [f"%{product_name}%"])
            else:
                # Query to fetch all products within allowed categories
                c.execute("""
                    SELECT P_ID, P_NAME, P_PRICE, P_STOCK_QTY, P_SKU, P_STATUS, P_CATEGORY, P_DATE_ADDED
                    FROM PRODUCT
                    WHERE P_CATEGORY IN ('Smartphone', 'Tablet', 'Laptop', 'Desktop', 'Accessory')
                    ORDER BY P_ID
                """)

            # Fetch rows and generate the table
            rows = c.fetchall()
            if not rows:
                products_table.value = "<p>No products found matching the criteria.</p>"
                total_products_label.value = "Total Products: 0"
                return

            # Generate the table HTML
            html = "<style>table {border-collapse: collapse; width: 100%;} td, th {border: 1px solid #ddd; padding: 8px;}</style>"
            html += "<table><tr><th>ID</th><th>Name</th><th>Price</th><th>Stock</th><th>SKU</th><th>Status</th><th>Category</th><th>Date Added</th></tr>"
            for index, row in enumerate(rows, start=1):  # Add an index column
                html += f"""
                <tr>
                    <td>{row[0]}</td>
                    <td>{row[1]}</td>
                    <td>${row[2]:.2f}</td>
                    <td>{row[3]}</td>
                    <td>{row[4]}</td>
                    <td>{row[5]}</td>
                    <td>{row[6]}</td>
                    <td>{row[7].strftime('%Y-%m-%d') if row[7] else ''}</td>
                </tr>
                """
            html += "</table>"
            products_table.value = html
            total_products_label.value = f"Total Products: {len(rows)}"
        except cx_Oracle.DatabaseError as e:
            error_obj, = e.args
            products_table.value = f"<p>❌ Database Error: {error_obj.message}</p>"
            total_products_label.value = "Total Products: 0"

    def on_product_refresh_clicked(b):
        search_product_id.value = 0
        search_product_name.value = ""
        load_products()

    def on_product_search_clicked(b):
        if search_product_id.value:
            load_products(product_id=search_product_id.value)
        elif search_product_name.value:
            load_products(product_name=search_product_name.value)
        else:
            product_search_status.value = "❗ Please enter either ID or Name to search."

    def on_view_product_back_button_clicked(b):
      showPage('Admin Menu')
    view_product_back_button.on_click(on_view_product_back_button_clicked)

    product_refresh_button.on_click(on_product_refresh_clicked)
    search_product_button.on_click(on_product_search_clicked)

    display(VBox([
        view_product_label,
        HBox([search_product_id, search_product_name, search_product_button]),
        HBox([product_refresh_button,view_product_back_button]),
        products_table,total_products_label,
        product_search_status
    ]))


########################################
########## Adm: Edit Product ##########
########################################
with tb.output_to('Adm: Edit Product'):
    edit_product_label = Label("Edit Product", style={'font_weight': 'bold', 'font_size': '18px'})
    edit_product_id = IntText(description="Product ID:", layout={'width': '350px', 'margin': '5px'})
    fetch_product_button = Button(description="Fetch Product", button_style='info', layout={'width': '150px', 'margin': '5px'})

    edit_product_name = Text(description="Product Name:", layout={'width': '350px', 'margin': '5px'})
    edit_product_price = FloatText(description="Price (HKD):", min=0, layout={'width': '350px', 'margin': '5px'})
    edit_product_description = Textarea(description="Description:", layout={'width': '350px', 'margin': '5px'})
    edit_product_color = Text(description="Color:", layout={'width': '350px', 'margin': '5px'})
    edit_product_capacity = Text(description="Capacity:", layout={'width': '350px', 'margin': '5px'})
    edit_product_stock = IntText(description="Stock Quantity:", min=0, layout={'width': '350px', 'margin': '5px'})
    edit_product_sku = Text(description="SKU:", disabled=True, layout={'width': '350px', 'margin': '5px'})
    edit_product_status = Dropdown(description="Status:", options=['Available', 'Out of Stock', 'Discontinued'], layout={'width': '350px', 'margin': '5px'})

    # Dropdown for Product Category
    edit_product_category = Dropdown(
        description="Category:",
        options=["Smartphone", "Tablet", "Laptop", "Desktop", "Accessory", "Other"],
        layout={'width': '350px', 'margin': '5px'}
    )

    update_product_button = Button(description="Update Product", button_style='success', disabled=True, layout={'width': '150px', 'margin': '5px'})
    clear_edit_product_button = Button(description="Clear Form", layout={'width': '150px', 'margin': '5px'})
    edit_product_status_label = Label(layout={'width': '1000px', 'margin': '5px'})
    edit_product_back_button = widgets.Button(description="Back", layout={'width': '150px', 'margin': '5px'})

    def on_fetch_product_clicked(b):
        try:
            product_id = edit_product_id.value
            if not product_id:
                edit_product_status_label.value = "❗ Please enter a Product ID."
                return

            c.execute("""
                SELECT P_NAME, P_PRICE, P_DESCRIPTION, P_COLOR, P_CAPACITY,
                       P_STOCK_QTY, P_SKU, P_STATUS, P_CATEGORY
                FROM PRODUCT WHERE P_ID = :1
            """, [product_id])

            product_data = c.fetchone()
            if not product_data:
                edit_product_status_label.value = "❗ No product found with this ID."
                update_product_button.disabled = True
                return

            # Populate fields with fetched data
            edit_product_name.value = product_data[0]
            edit_product_price.value = product_data[1]
            edit_product_description.value = product_data[2]
            edit_product_color.value = product_data[3]
            edit_product_capacity.value = product_data[4]
            edit_product_stock.value = product_data[5]
            edit_product_sku.value = product_data[6]
            edit_product_status.value = product_data[7]
            edit_product_category.value = product_data[8]
            update_product_button.disabled = False
            edit_product_status_label.value = "✅ Product data loaded. You can now update."

        except cx_Oracle.DatabaseError as e:
            error_obj, = e.args
            edit_product_status_label.value = f"❌ Database Error: {error_obj.message}"

    def on_update_product_clicked(b):
        try:
            # Check for blank fields
            if not all([edit_product_name.value, edit_product_description.value, edit_product_color.value,
                        edit_product_capacity.value, edit_product_sku.value]):
                edit_product_status_label.value = "❗ All fields are required."
                return

            # Check product name length
            if len(edit_product_name.value) < 2:
                edit_product_status_label.value = "❗ Product name must be at least 2 characters."
                return

            if len(edit_product_name.value) > 50:
                edit_product_status_label.value = "❗ Product name cannot exceed 50 characters."
                return

            # Check product price
            if edit_product_price.value <= 0:
                edit_product_status_label.value = "❗ Price must be greater than 0."
                return

            # Check product description length
            if len(edit_product_description.value) < 10:
                edit_product_status_label.value = "❗ Description must be at least 10 characters."
                return

            if len(edit_product_description.value) > 400:
                edit_product_status_label.value = "❗ Description cannot exceed 400 characters."
                return

            # Check product color length
            if len(edit_product_color.value) < 2:
                edit_product_status_label.value = "❗ Color must be at least 2 characters."
                return

            if len(edit_product_color.value) > 20:
                edit_product_status_label.value = "❗ Color cannot exceed 20 characters."
                return

            # Check product capacity length
            if len(edit_product_capacity.value) < 2:
                edit_product_status_label.value = "❗ Capacity must be at least 2 characters."
                return

            if len(edit_product_capacity.value) > 20:
                edit_product_status_label.value = "❗ Capacity cannot exceed 20 characters."
                return

            # Check stock quantity
            if edit_product_stock.value < 0:
                edit_product_status_label.value = "❗ Stock quantity cannot be negative."
                return

            # Update product in database
            c.execute("""
                UPDATE PRODUCT
                SET P_NAME = :1, P_PRICE = :2, P_DESCRIPTION = :3, P_COLOR = :4,
                    P_CAPACITY = :5, P_STOCK_QTY = :6, P_STATUS = :7, P_CATEGORY = :8
                WHERE P_ID = :9
            """, [
                edit_product_name.value, edit_product_price.value, edit_product_description.value,
                edit_product_color.value, edit_product_capacity.value, edit_product_stock.value,
                edit_product_status.value, edit_product_category.value, edit_product_id.value
            ])
            conn.commit()
            edit_product_status_label.value = "✅ Product information updated successfully."

        except cx_Oracle.DatabaseError as e:
            error_obj, = e.args
            edit_product_status_label.value = f"❌ Database Error: {error_obj.message}"

    def on_clear_edit_product_clicked(b):
        edit_product_id.value = 0
        edit_product_name.value = ""
        edit_product_price.value = 0
        edit_product_description.value = ""
        edit_product_color.value = ""
        edit_product_capacity.value = ""
        edit_product_stock.value = 0
        edit_product_sku.value = ""
        edit_product_status.value = "Available"
        edit_product_category.value = "Smartphone"
        update_product_button.disabled = True
        edit_product_status_label.value = ""

    def on_edit_product_back_button_clicked(b):
        showPage('Admin Menu')

    edit_product_back_button.on_click(on_edit_product_back_button_clicked)
    fetch_product_button.on_click(on_fetch_product_clicked)
    update_product_button.on_click(on_update_product_clicked)
    clear_edit_product_button.on_click(on_clear_edit_product_clicked)

    display(VBox([
        edit_product_label,
        edit_product_id,
        fetch_product_button,
        edit_product_name,
        edit_product_price,
        edit_product_description,
        edit_product_color,
        edit_product_capacity,
        edit_product_stock,
        edit_product_sku,
        edit_product_status,
        edit_product_category,  # Updated: Dropdown for category
        HBox([update_product_button, clear_edit_product_button, edit_product_back_button]),
        edit_product_status_label
    ]))

########################################
######## Adm: Proceed Order ###########
########################################
with tb.output_to('Adm: Proceed Order'):
    proceed_order_label = Label("Proceed Orders for Shipment", style={'font_weight': 'bold', 'font_size': '18px'})
    search_order_id = IntText(description="Order ID:", layout={'width': '350px', 'margin': '5px'})
    fetch_order_button = Button(description="Fetch Order", button_style='info', layout={'width': '150px', 'margin': '5px'})
    order_details = HTML(value="<p>Enter an Order ID to fetch details.</p>")
    proceed_status_label = Label(layout={'width': '350px', 'margin': '5px'})
    proceed_order_button = Button(description="Proceed Order", button_style='success', disabled=True, layout={'width': '150px', 'margin': '5px'})
    proceed_back_button = widgets.Button(description="Back", layout={'width': '150px', 'margin': '5px'})

    def fetch_order_details(b):
        try:
            if not search_order_id.value:
                proceed_status_label.value = "❗ Please enter a valid Order ID."
                return

            # Fetch order details
            c.execute("""
                SELECT OR_ID, OR_STATUS, OR_DATE, OR_SHIPDATE, OR_ESTIMATED_DAY, OR_REMARK
                FROM ORDERS WHERE OR_ID = :1
            """, [search_order_id.value])
            order = c.fetchone()

            if not order:
                order_details.value = "<p>No order found with this ID.</p>"
                proceed_order_button.disabled = True
                return

            order_details.value = f"""
            <style>table {{border-collapse: collapse; width: 100%;}} td, th {{border: 1px solid #ddd; padding: 8px;}}</style>
            <table>
                <tr><th>Order ID</th><td>{order[0]}</td></tr>
                <tr><th>Status</th><td>{order[1]}</td></tr>
                <tr><th>Order Date</th><td>{order[2].strftime('%Y-%m-%d')}</td></tr>
                <tr><th>Ship Date</th><td>{order[3].strftime('%Y-%m-%d')}</td></tr>
                <tr><th>Estimated Delivery</th><td>{order[4].strftime('%Y-%m-%d')}</td></tr>
                <tr><th>Remarks</th><td>{order[5]}</td></tr>
            </table>
            """
            proceed_order_button.disabled = order[1] != 'Pending'  # Enable button only if status is 'Pending'
            proceed_status_label.value = ""
        except cx_Oracle.DatabaseError as e:
            error_obj, = e.args
            order_details.value = f"<p>❌ Database Error: {error_obj.message}</p>"

    def proceed_order(b):
        try:
            c.execute("""
                UPDATE ORDERS SET OR_STATUS = 'Shipped', OR_SHIPDATE = SYSDATE
                WHERE OR_ID = :1
            """, [search_order_id.value])
            conn.commit()
            proceed_status_label.value = "✅ Order has been marked as 'Shipped'."
            fetch_order_details(None)  # Reload order details
        except cx_Oracle.DatabaseError as e:
            error_obj, = e.args
            proceed_status_label.value = f"❌ Database Error: {error_obj.message}"

    def proceed_back(b):
        showPage('Admin Menu')

    fetch_order_button.on_click(fetch_order_details)
    proceed_order_button.on_click(proceed_order)
    proceed_back_button.on_click(proceed_back)

    display(VBox([
        proceed_order_label,
        search_order_id,
        fetch_order_button,
        order_details,
        proceed_order_button,
        proceed_status_label,
        proceed_back_button
    ]))

########################################
####### Adm: Generate Report ##########
########################################
with tb.output_to('Adm: Generate Report'):
    report_label = Label("Generate Reports", style={'font_weight': 'bold', 'font_size': '18px'})
    report_type_dropdown = Dropdown(
        description="Report Type:",
        options=[
            "Sales Revenue (in dollars) by Date",
            "Sales Revenue (in pcs) by Date",
            "Total Trade-in Discount by Date",
            "List of Old Devices Received",
            "List of Out-of-Stock Products",
            "Top 10 Customers by Sales"
        ],
        layout={'width': '350px', 'margin': '5px'}
    )
    adm_generate_report_button = Button(description="Generate Report", button_style='primary', layout={'width': '200px', 'margin': '5px'})
    adm_report_output = HTML(value="<p>Select a report type and click 'Generate Report'.</p>")
    report_back_button = Button(description="Back", layout={'width': '150px', 'margin': '5px'})

    def adm_generate_report(b):
        try:
            report_type = report_type_dropdown.value

            if report_type == "Sales Revenue (in dollars) by Date":
                c.execute("""
                    SELECT TO_CHAR(OR_DATE, 'YYYY-MM-DD') AS SALE_DATE, SUM(OL_SUBTOTAL) AS REVENUE
                    FROM ORDERS
                    JOIN ORDERLINE ON ORDERS.OR_ID = ORDERLINE.ORDERS_OR_ID
                    GROUP BY TO_CHAR(OR_DATE, 'YYYY-MM-DD')
                    ORDER BY SALE_DATE
                """)
                rows = c.fetchall()
                if not rows:
                    adm_report_output.value = "<p>No data available for this report.</p>"
                    return

                html = "<style>table {border-collapse: collapse; width: 100%;} td, th {border: 1px solid #ddd; padding: 8px;}</style>"
                html += "<table><tr><th>Date</th><th>Revenue (in dollars)</th></tr>"
                for row in rows:
                    html += f"<tr><td>{row[0]}</td><td>${row[1]:.2f}</td></tr>"
                html += "</table>"
                adm_report_output.value = html

            elif report_type == "Sales Revenue (in pcs) by Date":
                c.execute("""
                    SELECT TO_CHAR(OR_DATE, 'YYYY-MM-DD') AS SALE_DATE, SUM(OL_P_QTY) AS TOTAL_UNITS
                    FROM ORDERS
                    JOIN ORDERLINE ON ORDERS.OR_ID = ORDERLINE.ORDERS_OR_ID
                    GROUP BY TO_CHAR(OR_DATE, 'YYYY-MM-DD')
                    ORDER BY SALE_DATE
                """)
                rows = c.fetchall()
                if not rows:
                    adm_report_output.value = "<p>No data available for this report.</p>"
                    return

                html = "<style>table {border-collapse: collapse; width: 100%;} td, th {border: 1px solid #ddd; padding: 8px;}</style>"
                html += "<table><tr><th>Date</th><th>Total Units Sold</th></tr>"
                for row in rows:
                    html += f"<tr><td>{row[0]}</td><td>{row[1]}</td></tr>"
                html += "</table>"
                adm_report_output.value = html

            elif report_type == "Total Trade-in Discount by Date":
                # Assuming trade-in discounts are stored in the `TR_PRICE` column of the TRADE_IN table
                c.execute("""
                    SELECT TO_CHAR(TR_DATE, 'YYYY-MM-DD') AS TRADE_DATE, SUM(TR_PRICE) AS TOTAL_DISCOUNT
                    FROM TRADE_IN
                    GROUP BY TO_CHAR(TR_DATE, 'YYYY-MM-DD')
                    ORDER BY TRADE_DATE
                """)
                rows = c.fetchall()
                if not rows:
                    adm_report_output.value = "<p>No data available for this report.</p>"
                    return

                html = "<table><tr><th>Date</th><th>Total Trade-in Discount (in dollars)</th></tr>"
                for row in rows:
                    html += f"<tr><td>{row[0]}</td><td>${row[1]:.2f}</td></tr>"
                html += "</table>"
                adm_report_output.value = html

            elif report_type == "List of Old Devices Received":
                c.execute("""
                    SELECT TR_ITEM AS DEVICE_NAME, TR_SERIAL_NUMBER AS SERIAL_NUMBER, TR_DATE AS RECEIVED_DATE
                    FROM TRADE_IN
                    ORDER BY TR_DATE
                """)
                rows = c.fetchall()
                if not rows:
                    adm_report_output.value = "<p>No data available for this report.</p>"
                    return

                html = "<table><tr><th>Device Name</th><th>Serial Number</th><th>Date Received</th></tr>"
                for row in rows:
                    html += f"<tr><td>{row[0]}</td><td>{row[1]}</td><td>{row[2]}</td></tr>"
                html += "</table>"
                adm_report_output.value = html

            elif report_type == "List of Out-of-Stock Products":
                c.execute("""
                    SELECT P_NAME, P_CATEGORY
                    FROM PRODUCT
                    WHERE P_STOCK_QTY = 0 AND P_CATEGORY IN ('Smartphone', 'Tablet', 'Laptop', 'Desktop', 'Accessory')
                    ORDER BY P_NAME
                """)
                rows = c.fetchall()
                if not rows:
                    adm_report_output.value = "<p>All products are in stock.</p>"
                    return

                html = "<table><tr><th>Product Name</th><th>Category</th></tr>"
                for row in rows:
                    html += f"<tr><td>{row[0]}</td><td>{row[1]}</td></tr>"
                html += "</table>"
                adm_report_output.value = html

            elif report_type == "Top 10 Customers by Sales":
                c.execute("""
                    SELECT M_FIRST_NAME || ' ' || M_LAST_NAME AS CUSTOMER_NAME, COUNT(ORDERS.OR_ID) AS TOTAL_ORDERS
                    FROM MEMBER
                    JOIN ORDERS ON MEMBER.M_ID = ORDERS.MEMBER_M_ID
                    GROUP BY M_FIRST_NAME, M_LAST_NAME
                    ORDER BY TOTAL_ORDERS DESC FETCH FIRST 10 ROWS ONLY
                """)
                rows = c.fetchall()
                if not rows:
                    adm_report_output.value = "<p>No customer data available for this report.</p>"
                    return

                html = "<table><tr><th>Customer Name</th><th>Total Orders</th></tr>"
                for row in rows:
                    html += f"<tr><td>{row[0]}</td><td>{row[1]}</td></tr>"
                html += "</table>"
                adm_report_output.value = html

        except cx_Oracle.DatabaseError as e:
            error_obj, = e.args
            adm_report_output.value = f"<p>❌ Database Error: {error_obj.message}</p>"

    def report_back(b):
        showPage('Admin Menu')

    adm_generate_report_button.on_click(adm_generate_report)
    report_back_button.on_click(report_back)

    display(VBox([
        report_label,
        report_type_dropdown,
        adm_generate_report_button,
        adm_report_output,
        report_back_button
    ]))


########################################
####### Adm: Manager Click Report ######
########################################
with tb.output_to('Adm: Manager Click Report'):
    from datetime import timedelta
    import matplotlib.pyplot as plt
    import pandas as pd
    from IPython.display import display
    from ipywidgets import IntText, Button, Text, VBox, HBox, Output

    report_min_clicks = IntText(description='Min Clicks:', value=0)

    report_generate_button = widgets.Button(description="Generate Report")
    report_clear_button = widgets.Button(description="Clear")
    report_back_button = widgets.Button(description="Back")
    report_output = widgets.Output()
    report_download_button = widgets.Button(description="Download CSV", disabled=True)
    report_filename = Text(description="Filename:", value="click_report.csv")

    def showTopClickRate(min_clicks = 0, from_index=1, to_index=2):
        c.execute("SELECT COUNT(*) FROM PRODUCT")
        for row in c:
          max_record_index = row[0]

        if max_record_index == 0:
          print("No record found.")
        else:
          print('\n4. Top 2 Products by Click Rate:')

        from_index = 1
        to_index = 2
        i = 0
        c.execute("""
        SELECT ROWNUM, P_ID, P_NAME, P_CLICKRATE
        FROM (
            SELECT P_ID, P_NAME, P_CLICKRATE
            FROM PRODUCT
            WHERE P_CLICKRATE >= :min_clicks AND P_CATEGORY IN('Accessory','Smartphone','Laptop')
            ORDER BY P_CLICKRATE DESC
                )
            """,{'min_clicks': min_clicks})
        top_product_ls = []
        for row in c:
          if from_index <= row[0] <= to_index:
            print('• '+ str(row[2])+ ' - '+ str(row[3])+' clicks')
            top_product_ls.append(str(row[2]))
            i = i + 1
            if i > 1:
              break
          else:
            continue


    def showBottomClickRate(min_clicks = 0):
        c.execute("SELECT COUNT(*) FROM PRODUCT")
        for row in c:
          max_record_index = row[0]

        if max_record_index == 0:
          print("No record found.")
        else:
          print('\n5. Bottom 1 Product by Click Rate:')

        c.execute("""
            SELECT P_ID, P_NAME, P_CLICKRATE
            FROM PRODUCT
            WHERE P_CLICKRATE >= :min_clicks AND P_CATEGORY IN('Accessory','Smartphone','Laptop')
            ORDER BY P_CLICKRATE
            """,{'min_clicks': min_clicks})
        bottom_pro_name = ''
        for row in c:
          print('• '+ str(row[1])+ ' - '+ str(row[2])+' clicks')
          bottom_pro_name = str(row[1])
          break

        print()
        print('Recommendations:')
        print(f"• Increase marketing for '{bottom_pro_name}'.")

    def get_rev_per_pro_cat(min_clicks = 0):
        c.execute('''SELECT ORDERLINE.PRODUCT_P_ID, SUM(ORDERLINE.OL_TOTALPRICE), SUM(ORDERLINE.OL_P_QTY) FROM PRODUCT INNER JOIN ORDERLINE ON PRODUCT.P_ID = ORDERLINE.PRODUCT_P_ID WHERE P_CATEGORY IN('Smartphone','Laptop','Accessory','Tablet','Desktop','Other') AND P_CLICKRATE >= :min_clicks GROUP BY ORDERLINE.PRODUCT_P_ID''',{'min_clicks': min_clicks})
        rev_product = c.fetchall()

        df = pd.DataFrame(rev_product, columns=['P_ID', 'Sales', 'Units']).sort_values(by='Sales', ascending = False)

        categories = []
        pro_names = []

        for product_id in df['P_ID']:
            c.execute('''SELECT P_CATEGORY, P_NAME FROM PRODUCT WHERE P_ID = :product_id''', {'product_id':product_id})
            category = c.fetchone()

            if category:
                categories.append(category[0])  # category is a tuple, take the first element
                pro_names.append(category[1])
            else:
                categories.append(None)
                pro_names.append(None)

        # Assign the categories to a new column in the DataFrame
        df['Category'] = categories
        df['Product Name'] = pro_names

        c.execute('''SELECT product.p_category, SUM(orderline.ol_totalprice) FROM PRODUCT INNER JOIN ORDERLINE ON PRODUCT.P_ID = orderline.product_p_id WHERE P_CATEGORY IN('Smartphone','Laptop','Accessory','Tablet','Desktop','Other') AND P_CLICKRATE >= :min_clicks GROUP BY product.p_category''', {'min_clicks': min_clicks})
        rev_cat = c.fetchall()
        df_2 = pd.DataFrame(rev_cat, columns=['Category', 'Total Sales (per category)']).sort_values(by='Total Sales (per category)', ascending = False)


        # Merge the product sales DataFrame with the category sales DataFrame
        merged_df = df.merge(df_2, on='Category', how='left', suffixes=('', '_Total'))

        # Calculate the percentage contribution of each product to its category
        merged_df['Percentage of Category'] = (merged_df['Sales'] / merged_df['Total Sales (per category)']) * 100
        merged_df['Percentage of Category'] = merged_df['Percentage of Category'].round(2).astype(str) + '%'



        # Display the final DataFrame with percentage contributions
        display(merged_df[['P_ID', 'Product Name', 'Sales', 'Category', 'Total Sales (per category)', 'Percentage of Category']].sort_values(by=['Total Sales (per category)', 'Category'], ascending=[False, True]).reset_index(drop=True))

        print('=== RECOMMENDATIONS AND INSIGHTS ===')

        # Get the top 2 categories
        top_category_by_sales = df_2.nlargest(2, 'Total Sales (per category)')['Category'].tolist()
        bottom_category_by_sales = df_2.nsmallest(1, 'Total Sales (per category)')['Category'].tolist()
        print(f"1. Focus on the most popular category :'{top_category_by_sales[0]}' by sales.")
        if bottom_category_by_sales != None and (bottom_category_by_sales[0] != top_category_by_sales[0]):
          print(f"   Comparing to other categories, the poor performer is: '{bottom_category_by_sales[0]}'.")

        # Filter the products for the top 2 categories
        best_sellers = df[df['Category'].isin(top_category_by_sales)]
        print(f'\n2. Best Seller in:')
        # Find the best-selling product(s) for each of the top categories
        for category in top_category_by_sales:
            best_selling_product = best_sellers[best_sellers['Category'] == category].nlargest(1, 'Sales')
            # Print the best-selling product(s) for the category
            print(f"\nCategory '{category}':")
            display(best_selling_product[['P_ID', 'Product Name']])

        print(f'\n3. Low revenue product:')
        poor_perf = df.nsmallest(1, 'Sales')['Product Name'].tolist()
        print(f"\nRecommendation: '{poor_perf[0]}' is the lowest revenue product - consider cut some budget for it.")

    def clickrate_and_sales_insights(min_clicks = 0):
        print('\n6. Star product by click rate and/ or sales:')
        c.execute('''
            SELECT PRODUCT.P_ID, PRODUCT.P_NAME, PRODUCT.P_CLICKRATE, SUM(ORDERLINE.OL_TOTALPRICE) AS Total_Sales
            FROM PRODUCT
            INNER JOIN ORDERLINE ON PRODUCT.P_ID = ORDERLINE.PRODUCT_P_ID
            WHERE P_CATEGORY IN ('Accessory', 'Smartphone', 'Laptop') AND P_CLICKRATE >= :min_clicks
            GROUP BY PRODUCT.P_ID, PRODUCT.P_NAME, PRODUCT.P_CLICKRATE
        ''', {'min_clicks':min_clicks})
        results = c.fetchall()

        df = pd.DataFrame(results, columns=['P_ID', 'Product Name', 'Total_Sales', 'Total_Click_Rate'])

        # Find the product with the highest sales
        highest_sales_product = df.nlargest(1, 'Total_Sales')

        # Find the product with the highest click rate
        highest_clickrate_product = df.nlargest(1, 'Total_Click_Rate')

        highest_sales_product_name = highest_sales_product['Product Name'].iloc[0]
        highest_clickrate_product_name = highest_clickrate_product['Product Name'].iloc[0]

        # Print the results
        if (highest_sales_product_name[0] == highest_clickrate_product_name[0]):
            print(f"• The star product among all categories should be {highest_sales_product_name} as it has the top sales and click rate. Keep it's promotion and marketing.")
        else:
            print(f"• The star product with best sales is '{highest_sales_product_name}', it generates the top sales. Can consider promote this product to boost sales.")
            print(f"• The star product with highest click rate is '{highest_clickrate_product_name}', it has the most click rate. Can consider provide some discounts or benefits when purchasing this product to boost sales.")

    def generate_report(b):
      with report_output:
        report_output.clear_output()
        report_download_button.disabled = True

        #start_date = report_start_date.value
        #end_date = report_end_date.value
        min_clicks = report_min_clicks.value

        # Validate dates
        #if not start_date or not end_date:
            #print("Error: Please select both start and end dates")
            #return
        #if start_date > end_date:
            #print("Error: Start date must be before end date")
            #return

        #print(f"Debug: Start Date = {start_date}, End Date = {end_date}")  # Debug log

        print(f"Generating report with minimum {min_clicks} clicks...")

        c.execute('''SELECT COUNT(*)
                          FROM PRODUCT
                          WHERE P_CLICKRATE >= :min_clicks AND P_CATEGORY IN('Accessory','Smartphone','Laptop')''', {'min_clicks': min_clicks})
        total_product_count = c.fetchone()
        if total_product_count is not None:
            # Extract the click rate as an integer
            total_product_count = int(total_product_count[0])  # Access the first element of the tuple
            print("Total Product Count:", total_product_count)
        else:
            print("No result found.")

        c.execute('''SELECT SUM(P_CLICKRATE)
                          FROM PRODUCT
                          WHERE P_CLICKRATE >= :min_clicks AND P_CATEGORY IN('Accessory','Smartphone','Laptop')''', {'min_clicks': min_clicks})
        total_clickrate = c.fetchone()
        if total_clickrate is not None:
            # Extract the click rate as an integer
            click_rate = int(total_clickrate[0])  # Access the first element of the tuple
            print("Click Rate:", click_rate)
        else:
            print("No result found.")


        c.execute('''SELECT AVG(P_CLICKRATE)
                          FROM PRODUCT
                          WHERE P_CLICKRATE >= :min_clicks AND P_CATEGORY IN('Accessory','Smartphone','Laptop')''', {'min_clicks': min_clicks})
        avg_click = c.fetchone()
        if avg_click is not None:
            # Extract the click rate as an integer
            avg_click = float(avg_click[0])  # Access the first element of the tuple
            print("Average Click Rate:", round(avg_click, 3))
        else:
            print("No result found.")

        try:
            query = """
            SELECT P_CATEGORY, P_ID, P_NAME, P_CLICKRATE
            FROM PRODUCT
            WHERE P_CLICKRATE >= :min_clicks AND P_CATEGORY IN('Accessory','Smartphone','Laptop')
            ORDER BY P_CLICKRATE DESC
            """

            c.execute(query, {'min_clicks': min_clicks})
            results = c.fetchall()

            if not results:
                print("No products match the criteria.")
                return

            # Create a DataFrame from the results
            df = pd.DataFrame(results, columns=['Product Category', 'Product ID', 'Product Name', 'Click Rate'])

            # Group by 'Product Category'
            grouped = df.groupby('Product Category')

            # Create a list to hold DataFrames for each category
            all_dataframes = []

            # Iterate through each category and create a sorted DataFrame
            for category, group in grouped:
                # Sort the group by 'Click Rate' in descending order
                sorted_group = group.sort_values(by='Click Rate', ascending=False)
                all_dataframes.append(sorted_group.reset_index(drop=True))

            # Concatenate all DataFrames into one
            final_df = pd.concat(all_dataframes, ignore_index=True)

            # Sort the final DataFrame by 'Click Rate' (descending) and 'Product Category' (ascending)
            final_df = final_df.sort_values(by=['Click Rate', 'Product Category','Product ID'], ascending=[False, True, True]).reset_index(drop=True)



            # Display insights header
            print(f"\n=== PRODUCT ANALYSIS  ===")
            print(f"• Total Products: {total_product_count}")
            print(f"• Minimum Click Threshold: {min_clicks}\n")

            # Display bar chart
            plt.figure(figsize=(12, 6))

            unique_categories = df['Product Category'].unique()
            colors = plt.cm.tab10.colors[:len(unique_categories)]
            color_map = dict(zip(unique_categories, colors))
            bar_colors = df['Product Category'].map(color_map)
            top_products = df.head(10)
            bars = plt.bar(df['Product Name'], df['Click Rate'], color=bar_colors)
            patches = [plt.Rectangle((0,0),1,1, color = color_map[cat]) for cat in unique_categories]
            plt.legend(patches, unique_categories, title='Product Categories')


            # df.head(10).plot.bar(x='Product Name', y='Click Rate', color='skyblue')
            plt.title('Top Products by Click Rate')
            plt.ylabel('Number of Clicks')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()

            # Display full data table
            print("\n=== COMPLETE PRODUCT CLICKRATE BY CATEGORY ===")
            display(final_df)

            if results:
              print('\n=== REVENUE PER PRODUCT FOR EACH CATEGORY ===')
              get_rev_per_pro_cat(min_clicks)
              showTopClickRate(min_clicks)
              print()
              showBottomClickRate(min_clicks)
              clickrate_and_sales_insights(min_clicks)



            # Enable download
            report_download_button.df = final_df
            report_download_button.disabled = False

        except Exception as e:
            print(f"Error generating report: {str(e)}")

    # What happens when "Download CSV" is clicked
    def download_report(b):
        if hasattr(b, 'df') and not report_download_button.disabled:
            filename = report_filename.value
            b.df.to_csv(filename, index=False)
            with report_output:
                print(f"Report saved as {filename}")
        else:
            with report_output:
                print("No data available to download")

    # What happens when "Clear" is clicked
    def clear_report(b):
        report_output.clear_output()
        report_download_button.disabled = True
        # report_start_date.value = date.today() - timedelta(days=30)
        # report_end_date.value = date.today()
        report_min_clicks.value = 0
        report_filename.value = "click_report.csv"

    # What happens when "Back" is clicked
    def go_back(b):
        showPage("Admin Menu")

    # Connect buttons
    report_generate_button.on_click(generate_report)
    report_download_button.on_click(download_report)
    report_clear_button.on_click(clear_report)
    report_back_button.on_click(go_back)

    # Layout
    #date_controls = widgets.HBox([report_start_date, report_end_date])
    filter_controls = widgets.HBox([report_min_clicks])
    button_row1 = widgets.HBox([report_generate_button, report_clear_button])
    button_row2 = widgets.HBox([report_download_button, report_filename])

    report_layout = widgets.VBox([
        widgets.Label("PRODUCT ANALYTICS DASHBOARD", style={'font_weight': 'bold'}),
        #date_controls,
        filter_controls,
        button_row1,
        report_output,
        button_row2,
        report_back_button
    ])

    display(report_layout)


########################################
########### Officer Tab #############
########################################
from google.colab.widgets import TabBar
import ipywidgets as widgets
from ipywidgets import Label, Text, Dropdown, Button, VBox, HBox, Password, Checkbox, HTML
from ipywidgets import interact, DatePicker, IntText, Layout, Box, GridspecLayout, Output, Textarea, FloatText
import re
import time
from datetime import date
import cx_Oracle
import random
import smtplib
from email.mime.text import MIMEText
from IPython.display import display, HTML
import json
from ipywidgets import HTML



# Email configuration (these credentials need to be replaced with valid ones for production)
EMAIL_FROM = "your_email@example.com"
EMAIL_PASSWORD = "your_email_password"
SMTP_SERVER = "smtp.example.com"
SMTP_PORT = 587

# Establish connection to the Oracle database
dsn_tns = cx_Oracle.makedsn(HOST_NAME, PORT_NUMBER, service_name=SERVICE_NAME)
conn = cx_Oracle.connect(user=USERNAME, password=PASSWORD, dsn=dsn_tns)
c = conn.cursor()


########################################
########### Officer Panel Tab #########
########################################
with tb.output_to('Officer Login'):
  # Store verification code and email for password reset
  verification_data = {
    'code': None,
    'email': None,
    'username': None
  }

  # Function to send verification email
  def send_verification_email(email, code):
    """Send password reset verification code to user's email"""
    try:
      msg = MIMEText(f"Your verification code is: {code}")
      msg['Subject'] = 'Password Reset Verification Code'
      msg['From'] = EMAIL_FROM
      msg['To'] = email

      with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.starttls()
        server.login(EMAIL_FROM, EMAIL_PASSWORD)
        server.send_message(msg)
      return True
    except Exception as e:
      print(f"Email sending error: {e}")
      return False

  # Officer Login Page
  login_label = Label("Officer Login", style={'font_weight': 'bold', 'font_size': '18px'})
  login_username = Text(description="Username:", placeholder="Enter your username",
                       layout={'width': '350px', 'margin': '5px'})
  login_password = Password(description="Password:", placeholder="Enter your password",
                          layout={'width': '350px', 'margin': '5px'})
  login_button = Button(description="Login", button_style='success',
                       layout={'width': '120px', 'margin': '5px'})
  forget_password_btn = Button(description="Forgot Password?", button_style='',
                              layout={'width': '150px', 'margin': '5px'})
  login_status = Label(layout={'width': '350px', 'margin': '5px'})

  login_page = VBox([
    login_label,
    login_username,
    login_password,
    HBox([login_button, forget_password_btn], layout={'justify_content': 'center', 'width': '350px'}),
    login_status
  ], layout={'align_items': 'center', 'justify_content': 'center', 'width': '100%', 'margin': '20px 0'})

  # Officer Main Page
  officer_main_label = Label("Officer Dashboard", style={'font_weight': 'bold', 'font_size': '18px'})
  welcome_label = Label(layout={'margin': '10px'})
  change_password_btn = Button(description="Change Password", button_style='primary',
                              layout={'width': '200px', 'margin': '10px'})

  view_tradein_btn = Button(description="View and Process Trade-ins",
                          button_style='info',
                          layout={'width': '200px', 'margin': '10px'})

  logout_button = Button(description="Logout", button_style='warning',
                        layout={'width': '200px', 'margin': '10px'})

  officer_main_page = VBox([
    officer_main_label,
    welcome_label,
    change_password_btn,
    view_tradein_btn,
    logout_button
  ], layout={'align_items': 'center', 'justify_content': 'center', 'width': '100%', 'margin': '20px 0'})

  # Change Password Page
  change_label = Label("Change Password", style={'font_weight': 'bold', 'font_size': '18px'})
  current_password = Password(description="Current PW:", placeholder="Enter current password",
                            layout={'width': '350px', 'margin': '5px'})
  new_password = Password(description="New PW:", placeholder="6-30 characters",
                         layout={'width': '350px', 'margin': '5px'})
  confirm_password = Password(description="Confirm PW:", placeholder="Re-enter new password",
                             layout={'width': '350px', 'margin': '5px'})
  change_button = Button(description="Change Password", button_style='primary',
                        layout={'width': '150px', 'margin': '5px'})
  change_status = Label(layout={'width': '350px', 'margin': '5px'})
  back_button_pw = Button(description="Back to Dashboard", button_style='',
                         layout={'width': '180px', 'margin': '5px'})

  change_password_page = VBox([
    change_label,
    current_password,
    new_password,
    confirm_password,
    change_button,
    change_status,
    back_button_pw
  ], layout={'align_items': 'center', 'justify_content': 'center', 'width': '100%', 'margin': '20px 0'})

  # Forgot Password Page - Step 1: Enter Email
  forgot_label = Label("Password Recovery", style={'font_weight': 'bold', 'font_size': '18px'})
  forgot_email = Text(description="Your Email:", placeholder="Enter your registered email",
                     layout={'width': '350px', 'margin': '5px'})
  send_code_button = Button(description="Send Verification Code", button_style='primary',
                           layout={'width': '200px', 'margin': '5px'})
  forgot_status = Label(layout={'width': '350px', 'margin': '5px'})
  back_to_login_btn = Button(description="Back to Login", button_style='',
                            layout={'width': '150px', 'margin': '5px'})

  forgot_page_step1 = VBox([
    forgot_label,
    forgot_email,
    send_code_button,
    forgot_status,
    back_to_login_btn
  ], layout={'align_items': 'center', 'justify_content': 'center', 'width': '100%', 'margin': '20px 0'})

  # Forgot Password Page - Step 2: Enter Verification Code
  verify_label = Label("Verify Your Identity", style={'font_weight': 'bold', 'font_size': '18px'})
  verify_code = Text(description="Verification Code:", placeholder="6-digit code from email",
                    layout={'width': '350px', 'margin': '5px'})
  new_password_reset = Password(description="New Password:", placeholder="6-30 characters",
                               layout={'width': '350px', 'margin': '5px'})
  confirm_password_reset = Password(description="Confirm Password:", placeholder="Re-enter new password",
                                   layout={'width': '350px', 'margin': '5px'})
  verify_button = Button(description="Reset Password", button_style='primary',
                        layout={'width': '150px', 'margin': '5px'})
  verify_status = Label(layout={'width': '350px', 'margin': '5px'})
  back_to_forgot_btn = Button(description="Back to Email Entry", button_style='',
                             layout={'width': '180px', 'margin': '5px'})

  forgot_page_step2 = VBox([
    verify_label,
    verify_code,
    new_password_reset,
    confirm_password_reset,
    verify_button,
    verify_status,
    back_to_forgot_btn
  ], layout={'align_items': 'center', 'justify_content': 'center', 'width': '100%', 'margin': '20px 0'})

  # Store logged-in username
  session_username = {"value": None}

  # Navigation functions for officer panel
  def show_login_page(btn=None):
      """Display the officer login page and hide all other sections"""
      # Hide all other sections
      login_page.layout.display = 'flex'  # Show only the login page
      officer_main_page.layout.display = 'none'
      change_password_page.layout.display = 'none'
      forgot_page_step1.layout.display = 'none'
      forgot_page_step2.layout.display = 'none'
      tradein_section.layout.display = 'none'

      # Reset login fields and verification data
      login_username.value = ""
      login_password.value = ""
      login_status.value = ""
      verification_data['code'] = None
      verification_data['email'] = None
      verification_data['username'] = None

  def show_officer_main(b=None):
    """Display the officer main dashboard and hide other sections"""
    try:
        # 1. Clear the contents of the output widgets
        tradein_output.clear_output()
        status_message.clear_output()

        # 2. Hide the entire trade-in section container
        tradein_section.layout.display = 'none'

        # 3. Show the officer dashboard and hide other pages
        login_page.layout.display = 'none'
        officer_main_page.layout.display = 'flex'  # Show dashboard
        change_password_page.layout.display = 'none'
        forgot_page_step1.layout.display = 'none'
        forgot_page_step2.layout.display = 'none'

        # 4. Update welcome message
        welcome_label.value = f"Welcome, {session_username['value']}"

    except Exception as e:
        print(f"Error switching to dashboard: {str(e)}")

  def show_change_password(b=None):
    """Display the change password page and hide other sections"""
    login_page.layout.display = 'none'
    officer_main_page.layout.display = 'none'
    change_password_page.layout.display = 'flex'
    forgot_page_step1.layout.display = 'none'
    forgot_page_step2.layout.display = 'none'
    current_password.value = ""
    new_password.value = ""
    confirm_password.value = ""
    change_status.value = ""

  def show_forgot_step1(b=None):
    """Display the first step of password recovery (email entry) and hide other sections"""
    login_page.layout.display = 'none'
    officer_main_page.layout.display = 'none'
    change_password_page.layout.display = 'none'
    forgot_page_step1.layout.display = 'flex'
    forgot_page_step2.layout.display = 'none'
    forgot_email.value = ""
    forgot_status.value = ""

  def show_forgot_step2(b=None):
    """Display the second step of password recovery (code verification) and hide other sections"""
    login_page.layout.display = 'none'
    officer_main_page.layout.display = 'none'
    change_password_page.layout.display = 'none'
    forgot_page_step1.layout.display = 'none'
    forgot_page_step2.layout.display = 'flex'
    verify_code.value = ""
    new_password_reset.value = ""
    confirm_password_reset.value = ""
    verify_status.value = ""

  # Handler for login button
  def on_login_button_clicked(b):
    """Process officer login with validation"""
    try:
      if not login_username.value or not login_password.value:
        login_status.value = "❗ Both username and password are required."
        return

      # First check if username exists
      c.execute("SELECT COUNT(*) FROM STAFF WHERE S_USERNAME = :1 AND S_TYPE = 'Officer'",
                [login_username.value])
      if c.fetchone()[0] == 0:
        login_status.value = "❌ Username not found."
        return

      # Then check password and active status
      c.execute("""
        SELECT S_USERNAME, S_STATUS FROM STAFF
        WHERE S_USERNAME = :1 AND S_PW = :2 AND S_TYPE = 'Officer'
      """, [login_username.value, login_password.value])

      result = c.fetchone()
      if result:
        if result[1] != 'Active':
          login_status.value = "❌ Your account is inactive. Please contact an administrator."
          return

        session_username["value"] = result[0]
        login_status.value = "✅ Login successful"
        show_officer_main()
      else:
        login_status.value = "❌ Password incorrect."
    except cx_Oracle.DatabaseError as e:
      error_obj, = e.args
      login_status.value = f"❌ Database Error: {error_obj.message}"

  # Handler for change password button
  def on_change_button_clicked(b):
    """Process password change request with validation"""
    try:
      if not session_username["value"]:
        change_status.value = "❗ Please login first."
        return

      if not current_password.value or not new_password.value or not confirm_password.value:
        change_status.value = "❗ All fields are required."
        return

      if new_password.value != confirm_password.value:
        change_status.value = "❗ New passwords do not match."
        return

      if not is_valid_password(new_password.value):
        change_status.value = "❗ Password must be 6-30 characters."
        return

      # Verify current password
      c.execute("SELECT COUNT(*) FROM STAFF WHERE S_USERNAME = :1 AND S_PW = :2",
                [session_username["value"], current_password.value])
      if c.fetchone()[0] == 0:
        change_status.value = "❗ Current password is incorrect."
        return

      c.execute("UPDATE STAFF SET S_PW = :1 WHERE S_USERNAME = :2",
                [new_password.value, session_username["value"]])
      conn.commit()
      change_status.value = "✅ Password updated successfully."
      current_password.value = ""
      new_password.value = ""
      confirm_password.value = ""
    except cx_Oracle.DatabaseError as e:
      error_obj, = e.args
      change_status.value = f"❌ Database Error: {error_obj.message}"

  # Handler for sending verification code
  def on_send_code_clicked(b):
    """Process request to send password reset verification code"""
    try:
      email = forgot_email.value.strip()
      if not email:
        forgot_status.value = "❗ Please enter your email address."
        return

      if not is_valid_email(email):
        forgot_status.value = "❗ Invalid email format."
        return

      # Check if email exists in database
      c.execute("SELECT S_USERNAME, S_EMAIL FROM STAFF WHERE S_EMAIL = :1 AND S_TYPE = 'Officer'", [email])
      result = c.fetchone()

      if not result:
        forgot_status.value = "❗ No officer account found with this email."
        return

      username, registered_email = result
      verification_data['username'] = username
      verification_data['email'] = registered_email
      verification_data['code'] = str(random.randint(100000, 999999))

      # For demo purposes, we'll just show the code in the status
      # In production, this would be sent via email
      if send_verification_email(email, verification_data['code']):
        forgot_status.value = f"✅ Verification code sent to {email}. (Demo: Code is {verification_data['code']})"
        show_forgot_step2()
      else:
        forgot_status.value = "❌ Failed to send verification code. Please try again."
    except cx_Oracle.DatabaseError as e:
      error_obj, = e.args
      forgot_status.value = f"❌ Database Error: {error_obj.message}"

  # Handler for verifying code and resetting password
  def on_verify_code_clicked(b):
    """Process password reset with verification code"""
    try:
      if not verify_code.value or not new_password_reset.value or not confirm_password_reset.value:
        verify_status.value = "❗ All fields are required."
        return

      if verify_code.value != verification_data['code']:
        verify_status.value = "❗ Invalid verification code."
        return

      if new_password_reset.value != confirm_password_reset.value:
        verify_status.value = "❗ Passwords do not match."
        return

      if not is_valid_password(new_password_reset.value):
        verify_status.value = "❗ Password must be 6-30 characters."
        return

      # Update password in database
      c.execute("UPDATE STAFF SET S_PW = :1 WHERE S_USERNAME = :2",
                [new_password_reset.value, verification_data['username']])
      conn.commit()
      verify_status.value = "✅ Password reset successfully. Please login with your new password."

      # Clear verification data
      verification_data['code'] = None
      verification_data['email'] = None
      verification_data['username'] = None

      # Return to login page after delay
      import time
      time.sleep(2)
      show_login_page()
    except cx_Oracle.DatabaseError as e:
      error_obj, = e.args
      verify_status.value = f"❌ Database Error: {error_obj.message}"

  def on_logout_button_clicked(b):
    """Process logout by clearing session data"""
    session_username["value"] = None
    showPage("Officer Login")

    login_page.layout.display = 'flex'
    officer_main_page.layout.display = 'none'
    change_password_page.layout.display = 'none'
    forgot_page_step1.layout.display = 'none'
    forgot_page_step2.layout.display = 'none'
    tradein_section.layout.display = 'none'
    login_username.value = ""
    login_password.value = ""
    login_status.value = ""
    verification_data['code'] = None
    verification_data['email'] = None
    verification_data['username'] = None


########################################
########### Trade-in Section ###########
########################################


  ### View Trade-in Items Section
  tradein_label = Label(
    "Trade-in Processing",
    style={
        'font_weight': 'bold',
        'font_size': '18px',
        'color': '#2c3e50',
        'margin': '10px 0'
    }
  )

  # Use an Output widget with better styling
  tradein_output = Output(
    layout={
        'width': '95%',
        'height': '400px',
        'overflow': 'auto',
        'border': '1px solid #e0e0e0',
        'border_radius': '5px',
        'padding': '10px',
        'margin': '10px 0'
    }
  )

  # Status message with better styling
  status_message = Output(
    layout={
        'width': '95%',
        'margin': '10px 0'
    }
  )

  back_button_new = Button(
    description='Back to Officer Dashboard',
    icon='reply',
    layout={
        'width': 'auto',
        'height': '32px',
        'margin': '4px 0',
        'padding': '0 12px',
      },
    style={
        'button_color': '#6c757d',
        'font_weight': 'bold',
        'text_color': 'white',
        'border': '1px solid #5a6268',
        'font_size': '13px',
        'line_height': '30px'
      }
  )


  # Styled container for the section
  tradein_section = VBox([
    tradein_label,
    tradein_output,
    back_button_new,
    status_message
  ], layout={
    'align_items': 'center',
    'width': '100%',
    'padding': '15px',
    'border': '1px solid #eee',
    'border_radius': '8px',
    'box_shadow': '0 2px 4px rgba(0,0,0,0.1)'
  })


  # Function to process trade-in actions
  def display_tradein_items(items=None, show_only_pending=True, search_member_id=None, search_order_id=None):
    try:
        if items is None:
            base_query = """
                SELECT t.TR_ID, p.P_NAME,
                       TO_CHAR(t.TR_DATE, 'YYYY-MM-DD HH24:MI') as TR_DATE,
                       t.TR_PRICE, m.M_ID, o.OR_ID, t.TR_STATUS
                FROM TRADE_IN t
                JOIN ORDERS o ON t.ORDERS_OR_ID = o.OR_ID
                JOIN ORDERLINE ol ON o.OR_ID = ol.OL_ID
                JOIN PRODUCT p ON ol.PRODUCT_P_ID = p.P_ID
                JOIN MEMBER m ON o.MEMBER_M_ID = m.M_ID
                {where_clause}
                ORDER BY t.TR_DATE DESC
            """

            where_clauses = []
            if show_only_pending:
                where_clauses.append("t.TR_STATUS = 'Pending'")
            if search_member_id:
                where_clauses.append(f"m.M_ID = {search_member_id}")
            if search_order_id:
                where_clauses.append(f"o.OR_ID = {search_order_id}")

            where_clause = "WHERE " + " AND ".join(where_clauses) if where_clauses else ""
            final_query = base_query.format(where_clause=where_clause)

            c.execute(final_query)
            items = c.fetchall()


        # Clear previous widgets
        tradein_output.clear_output()

        with tradein_output:
            # Create search controls at the top
            search_controls = widgets.HBox([
                widgets.Label(value="Search by:"),
                widgets.IntText(placeholder="Member ID", description="Member ID:", layout=widgets.Layout(width='200px')),
                widgets.IntText(placeholder="Order ID", description="Order ID:", layout=widgets.Layout(width='200px')),
                widgets.Button(description="Search", button_style='primary'),
                widgets.Button(description="Reset", button_style='warning')
            ])

            def handle_search(btn):
                member_id = search_controls.children[1].value
                order_id = search_controls.children[2].value
                display_tradein_items(
                    show_only_pending=show_only_pending,
                    search_member_id=member_id if member_id else None,
                    search_order_id=order_id if order_id else None
                )

            def handle_reset(btn):
                search_controls.children[1].value = 0
                search_controls.children[2].value = 0
                display_tradein_items()

            search_controls.children[3].on_click(handle_search)
            search_controls.children[4].on_click(handle_reset)

            # Create a container for all trade-in rows
            tradein_container = widgets.VBox()
            rows = [search_controls]  # Add search controls at the top

            # Header row with improved styling
            header = widgets.HBox([
                widgets.Label(value="TradeIn ID", layout=widgets.Layout(width='100px', margin='0 5px')),
                widgets.Label(value="Product Name", layout=widgets.Layout(width='200px', margin='0 5px')),
                widgets.Label(value="Date", layout=widgets.Layout(width='180px', margin='0 5px')),
                widgets.Label(value="Price", layout=widgets.Layout(width='100px', margin='0 5px')),
                widgets.Label(value="Member ID", layout=widgets.Layout(width='100px', margin='0 5px')),
                widgets.Label(value="Order ID", layout=widgets.Layout(width='100px', margin='0 5px')),
                widgets.Label(value="Status", layout=widgets.Layout(width='120px', margin='0 5px')),
                widgets.Label(value="Actions", layout=widgets.Layout(width='300px', margin='0 5px'))
            ])
            header.add_class("tradein-header")
            header.style = {
                'font_weight': 'bold',
                'background_color': '#f8f9fa',
                'padding': '8px',
                'border_bottom': '2px solid #dee2e6'
            }
            rows.append(header)

            if not items:
                rows.append(widgets.HTML("<p style='padding: 16px; color: #666;'>No trade-in items found</p>"))
            else:
                for item in items:
                    tr_id, prod_name, tr_date, tr_price, m_id, order_id, tr_status = item[:7]
                    formatted_price = f"${tr_price:,.2f}" if tr_price else "$0.00"
                    status_color = {
                        'Pending': 'orange',
                        'Accepted': 'green',
                        'Rejected': 'red',
                        'Bypassed': 'gray'
                    }.get(tr_status, 'black')

                    # Create action buttons
                    accept_btn = widgets.Button(
                        description="Accept",
                        disabled=(tr_status != 'Pending'),
                        layout=widgets.Layout(width='100px')
                    )
                    cancel_btn = widgets.Button(
                        description="Cancel",
                        disabled=(tr_status != 'Pending'),
                        layout=widgets.Layout(width='100px')
                    )
                    proceed_btn = widgets.Button(
                        description="Proceed",
                        disabled=(tr_status != 'Pending'),
                        layout=widgets.Layout(width='140px')
                    )

                    # Store trade-in ID in button attributes
                    accept_btn.tr_id = tr_id
                    cancel_btn.tr_id = tr_id
                    proceed_btn.tr_id = tr_id

                    # Create row
                    row = widgets.HBox([
                        widgets.Label(value=str(tr_id), layout=widgets.Layout(width='100px')),
                        widgets.Label(value=prod_name, layout=widgets.Layout(width='200px')),
                        widgets.Label(value=tr_date, layout=widgets.Layout(width='180px')),
                        widgets.Label(value=formatted_price, layout=widgets.Layout(width='100px')),

                        widgets.Label(value=str(m_id), layout=widgets.Layout(width='100px'),style={'align_items': 'center', 'justify_content': 'center'}),
                        widgets.Label(value=str(order_id), layout=widgets.Layout(width='100px'),style={'align_items': 'center', 'justify_content': 'center'}),
                        widgets.Label(value=tr_status, style={'font_weight': 'bold', 'color': status_color},
                                    layout=widgets.Layout(width='120px')),
                        widgets.HBox([accept_btn, cancel_btn, proceed_btn],
                                    layout=widgets.Layout(width='300px'))
                    ])
                    row.add_class("tradein-row")
                    rows.append(row)

                    # Set button click handlers
                    accept_btn.on_click(lambda btn, action='accept': handle_tradein_action(btn, action))
                    cancel_btn.on_click(lambda btn, action='cancel': handle_tradein_action(btn, action))
                    proceed_btn.on_click(lambda btn, action='proceed': handle_tradein_action(btn, action))

            tradein_container.children = rows
            display(tradein_container)

    except Exception as e:
        tradein_output.clear_output()
        with tradein_output:
            display(HTML(f"<p style='color: red;'>❌ Error: {str(e)}</p>"))

  def handle_tradein_action(btn, action=None):
    try:
        # If called from button click, btn will be the button widget
        if isinstance(btn, widgets.Button):
            tr_id = btn.tr_id
            if action is None:  # Handle case where action is passed directly
                action = btn.action
        else:  # Handle case where tr_id is passed directly
            tr_id = btn
            action = action

        tr_id = int(tr_id)

        # First get the order number for the message
        c.execute("""
            SELECT t.ORDERS_OR_ID, t.TR_STATUS
            FROM trade_in t
            WHERE t.TR_ID = :1
            ORDER BY t.ORDERS_OR_ID
        """, [tr_id])
        order_info = c.fetchone()
        order_id = order_info[0]
        current_status = order_info[1]

        if action == 'accept':
            c.execute("UPDATE trade_in SET TR_STATUS = 'Accepted' WHERE TR_ID = :1", [tr_id])
            message = f"✅ Trade-in request for Order #{order_id} has been accepted"
        elif action == 'cancel':
            c.execute("UPDATE orders SET OR_STATUS = 'Canceled' WHERE OR_ID = (SELECT ORDERS_OR_ID FROM trade_in WHERE TR_ID = :1)", [tr_id])
            c.execute("UPDATE trade_in SET TR_STATUS = 'Rejected' WHERE TR_ID = :1", [tr_id])
            message = f"❌ Trade-in request for Order #{order_id} has been canceled"
        elif action == 'proceed':
            c.execute("UPDATE trade_in SET TR_STATUS = 'Completed' WHERE TR_ID = :1", [tr_id])
            message = f"✔️ Trade-in request for Order #{order_id} has been completed"

        conn.commit()

        # Display confirmation message
        tradein_output.clear_output()
        with tradein_output:
            display(HTML(f"<p style='color: green; font-weight: bold;'>{message}</p>"))
            # Wait a moment before refreshing
            time.sleep(2)  # You may need to import time at the top of your notebook

        # Refresh the display showing only pending items
        display_tradein_items(show_only_pending=True)

    except Exception as ex:
        tradein_output.clear_output()
        with tradein_output:
            display(HTML(f"<p style='color: red;'>❌ Failed to update trade-in: {str(ex)}</p>"))



  def show_login_page(btn=None):
    """Show the login page and hide other sections"""
    login_page.layout.display = 'none'
    officer_main_page.layout.display = 'flex'
    change_password_page.layout.display = 'none'
    forgot_page_step1.layout.display = 'none'
    forgot_page_step2.layout.display = 'none'
    tradein_section.layout.display = 'none'


  # Set up button actions
  login_button.on_click(on_login_button_clicked)
  change_button.on_click(on_change_button_clicked)
  change_password_btn.on_click(show_change_password)
  back_button_pw.on_click(show_officer_main)
  logout_button.on_click(on_logout_button_clicked)
  forget_password_btn.on_click(show_forgot_step1)
  back_to_login_btn.on_click(show_login_page)
  send_code_button.on_click(on_send_code_clicked)
  verify_button.on_click(on_verify_code_clicked)
  back_to_forgot_btn.on_click(show_forgot_step1)


  # Initial display
  login_page.layout.display = 'flex'
  officer_main_page.layout.display = 'none'
  change_password_page.layout.display = 'none'
  forgot_page_step1.layout.display = 'none'
  forgot_page_step2.layout.display = 'none'
  tradein_section.layout.display = 'none'

  display(VBox([
    login_page,
    officer_main_page,
    change_password_page,
    forgot_page_step1,
    forgot_page_step2
  ], layout={'width': '100%'}))

with tb.output_to('Officer: View and Process TradeIns'):
  def show_tradein_interface(btn=None):
    """Show the trade-in processing section"""
    try:
        # 1. Switch to the "View and Process TradeIns" tab
        tab_index = tb.tab_names.index('Officer: View and Process TradeIns')
        tb.selected_index = tab_index
        # 1. Hide other pages first
        login_page.layout.display = 'none'
        officer_main_page.layout.display = 'none'  # Hide dashboard
        change_password_page.layout.display = 'none'
        forgot_page_step1.layout.display = 'none'
        forgot_page_step2.layout.display = 'none'

        # 2. Show the trade-in section
        tradein_section.layout.display = 'flex'

        # 3. Refresh the trade-in items display
        with tb.output_to('Officer: View and Process TradeIns'):
            tradein_output.clear_output()
            display_tradein_items()  # Your existing function
        display_tradein_items()  # Your existing function

    except Exception as e:
        status_message.clear_output()
        with status_message:
            print(f"Error showing trade-in interface: {str(e)}")

  def show_officer_secondary(b=None):
    """Display the officer main dashboard and hide other sections"""
    try:
        # 1. Clear the contents of the output widgets
        tradein_output.clear_output()
        status_message.clear_output()
        #tab_index = tb_ui.tab_names.index('Officer Login')
        #tb_ui.selected_index = tab_index
        showPage("Officer Login")
        # 2. Hide the entire trade-in section container
        tradein_section.layout.display = 'none'

        # 3. Show the officer dashboard and hide other pages
        login_page.layout.display = 'none'
        officer_main_page.layout.display = 'flex'  # Show dashboard
        change_password_page.layout.display = 'none'
        forgot_page_step1.layout.display = 'none'
        forgot_page_step2.layout.display = 'none'

        # 4. Update welcome message
        welcome_label.value = f"Welcome, {session_username['value']}"

    except Exception as e:
        print(f"Error switching to dashboard: {str(e)}")


  def show_officer_interface(btn=None):
      """Show the trade-in processing section"""
      try:
          # 1. Switch to the "View and Process TradeIns" tab
          tab_index = tb.tab_names.index('Officer Login')
          tb.selected_index = tab_index
          # 1. Hide other pages first
          login_page.layout.display = 'none'
          officer_main_page.layout.display = 'flex'  # Hide dashboard
          change_password_page.layout.display = 'none'
          forgot_page_step1.layout.display = 'none'
          forgot_page_step2.layout.display = 'none'

      except Exception as e:
          status_message.clear_output()
          with status_message:
              print(f"Error showing trade-in interface: {str(e)}")


  back_button_new.on_click(show_officer_secondary)
  view_tradein_btn.on_click(show_tradein_interface)
  tradein_section.layout.display = 'flex'

  display(VBox([
    tradein_section
  ], layout={'width': '100%'}))

### End ####

showPage('Welcome')

Database connection established successfully.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#example officer: angeltest, 123456